<h1>Data Streaming Algorithms</h1>

<h2>HW1</h2>

Slava Salasin, 319238119
TODO - Bayan

<h2>Dry Part

<h3>Morris algorithm

In order to

<h3>Reservoir sampling

In class we proved the Reservoir sampling algorithm for $j>k$. Now we will provide a supplementary proof for $j<=k$.
<br>
$P[a_{j}\ will\ be\ sampled] =_{1} P[a_{j}\ will\ not\ be\ replaced\ on\ j=k+1] \cdot  ... \cdot P[a_{j}\ will\ not\ be\ replaced\ on\ j=n] =_{2} (1 - P[a_{j}\ will\ be\ replaced\ on\ j=k+1]) \cdot  ... \cdot (1 - P[a_{j}\ will\ be\ replaced\ on\ j=n]) =_{3} (1 - \dfrac{1}{k+1}) \cdot  ... \cdot (1 - \dfrac{1}{n}) =_{4} \dfrac{k}{k+1} \cdot  ... \cdot \dfrac{n-1}{n} =_{5} \dfrac{k}{n}$
<br>
1. If $a_{j}$ is sampled, then it's not replaced in iteration k+1 and k+2 etc.
2. Probability complement.
3. $P[a_{j}\ will\ be\ replaced\ on\ j=k+1] = \dfrac{k}{k+1} \cdot \dfrac{1}{k}$

    $\dfrac{k}{k+1}$ - The probability to sample an element from the stream is the sample size divided by the element index. $\dfrac{1}{k}$ - The probabiliyy to replace a specific element from the sample is 1 divided by the size of the sample.

4. Simple algebra.
5. Simple algebra.

In [33]:
import random
import numpy as np

random.seed(42)

In [34]:
def fm_alpha():
    pass

In [35]:
def fm_beta():
    pass

In [36]:
def fm_final():
    pass

In [37]:
stream = []
for i in range(1, 10001):
    count = random.sample(range(1, 501), k=1)
    stream.extend(i for j in range(count[0]))

u, counts = np.unique(stream, return_counts=True)
counts

array([328,  58,  13, ..., 147, 142, 273])